<a href="https://colab.research.google.com/github/kusawokoshitemori/gan_colab/blob/main/mnist_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

In [2]:
# ハイパーパラメータの設定
EPOCHS = 50
BATCH_SIZE = 64
LATENT_DIM = 100 # ノイズベクトルの次元
LR_G = 0.0002 # Generatorの学習率
LR_D = 0.0002 # Discriminatorの学習率

In [3]:
# 画像の保存先ディレクトリ
SAVE_DIR = 'generated_images'
os.makedirs(SAVE_DIR, exist_ok=True)

In [4]:
# データの準備
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # -1から1に正規化
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

100%|██████████| 9.91M/9.91M [00:10<00:00, 905kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.49MB/s]


In [5]:
# Generatorの定義
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # 入力: latent_dim次元のノイズ
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 28 * 28), # MNIST画像のサイズ
            nn.Tanh() # 出力を-1から1にクリップ
        )

    def forward(self, input):
        return self.main(input).view(-1, 1, 28, 28) # 1チャンネルの28x28画像にreshape

In [6]:
# Discriminatorの定義
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # 入力: 1チャンネルの28x28画像
            nn.Linear(28 * 28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid() # 0から1の確率を出力
        )

    def forward(self, input):
        return self.main(input.view(-1, 28 * 28))

In [7]:
# モデルのインスタンス化
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(LATENT_DIM).to(device)
discriminator = Discriminator().to(device)

In [8]:
# 損失関数とOptimizer
criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer_g = optim.Adam(generator.parameters(), lr=LR_G, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=LR_D, betas=(0.5, 0.999))

In [9]:
# 学習ループ
print("学習を開始します...")
for epoch in range(EPOCHS):
    for i, (real_images, _) in enumerate(train_loader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)

        # Discriminatorの学習
        discriminator.zero_grad()

        # 本物画像
        real_labels = torch.ones(batch_size, 1).to(device)
        output_real = discriminator(real_images)
        loss_d_real = criterion(output_real, real_labels)

        # 偽物画像
        noise = torch.randn(batch_size, LATENT_DIM).to(device)
        fake_images = generator(noise)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        output_fake = discriminator(fake_images.detach()) # Generatorの勾配を伝播させない
        loss_d_fake = criterion(output_fake, fake_labels)

        loss_d = loss_d_real + loss_d_fake
        loss_d.backward()
        optimizer_d.step()

        # Generatorの学習
        generator.zero_grad()
        output_g = discriminator(fake_images) # Discriminatorが本物と判断するように学習
        loss_g = criterion(output_g, real_labels) # Generatorは偽物画像を本物と誤認させたいので、本物ラベルと比較

        loss_g.backward()
        optimizer_g.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{EPOCHS}], Step [{i+1}/{len(train_loader)}], D_loss: {loss_d.item():.4f}, G_loss: {loss_g.item():.4f}")

    # 5エポックごとに画像を生成して保存
    if (epoch + 1) % 5 == 0:
        with torch.no_grad():
            fixed_noise = torch.randn(64, LATENT_DIM).to(device) # 一貫したノイズで生成
            generated_samples = generator(fixed_noise).cpu()
            save_image(generated_samples, os.path.join(SAVE_DIR, f'epoch_{epoch+1:03d}.png'), nrow=8, normalize=True)
print("学習が完了しました")

学習を開始します...
Epoch [1/50], Step [100/938], D_loss: 1.4428, G_loss: 0.6850
Epoch [1/50], Step [200/938], D_loss: 1.3115, G_loss: 0.7806
Epoch [1/50], Step [300/938], D_loss: 1.3479, G_loss: 1.0446
Epoch [1/50], Step [400/938], D_loss: 1.4496, G_loss: 0.9418
Epoch [1/50], Step [500/938], D_loss: 1.0714, G_loss: 1.0666
Epoch [1/50], Step [600/938], D_loss: 0.8880, G_loss: 1.6554
Epoch [1/50], Step [700/938], D_loss: 0.8832, G_loss: 2.2415
Epoch [1/50], Step [800/938], D_loss: 0.9410, G_loss: 1.4081
Epoch [1/50], Step [900/938], D_loss: 0.9913, G_loss: 1.9709
Epoch [2/50], Step [100/938], D_loss: 0.9867, G_loss: 1.6307
Epoch [2/50], Step [200/938], D_loss: 1.0142, G_loss: 2.3971
Epoch [2/50], Step [300/938], D_loss: 0.7251, G_loss: 1.7615
Epoch [2/50], Step [400/938], D_loss: 0.6790, G_loss: 2.0292
Epoch [2/50], Step [500/938], D_loss: 0.7396, G_loss: 2.3628
Epoch [2/50], Step [600/938], D_loss: 0.9230, G_loss: 3.4057
Epoch [2/50], Step [700/938], D_loss: 0.8701, G_loss: 1.5397
Epoch [2/50]